In [56]:
import wget
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import linear_model

# Reading Data
train_dir = wget.download("https://s3.amazonaws.com/istreet-questions-us-east-1/443606/census.csv")
df = pd.read_csv(train_dir, header = None)

cols = []
for i in range(0, len(df.columns)):
    cols.append(df.iloc[0,i].split('=')[0])
    df.iloc[:, i] = [k[1] for k in df.iloc[:,i].str.split('=')]
df.columns = cols

In [57]:
df.head()

,age,sex,education,native-country,race,marital-status,workclass,occupation,hours-per-week,income,capital-gain,capital-loss
0,Middle-aged,Male,Bachelors,United-States,White,Never-married,State-gov,Adm-clerical,Full-time,Small,Low,None
1,Senior,Male,Bachelors,United-States,White,Married-civ-spouse,Self-emp-not-inc,Exec-managerial,Part-time,Small,None,None
2,Middle-aged,Male,HS-grad,United-States,White,Divorced,Private,Handlers-cleaners,Full-time,Small,None,None
3,Senior,Male,11th,United-States,Black,Married-civ-spouse,Private,Handlers-cleaners,Full-time,Small,None,None
4,Middle-aged,Female,Bachelors,Cuba,Black,Married-civ-spouse,Private,Prof-specialty,Full-time,Small,None,None


In [58]:
rules = ['{native-country=United-States,capital-gain=None}=>{capital-loss=None}',
'{native-country=United-States,capital-loss=None}=>{capital-gain=None}',
'{capital-gain=None,capital-loss=None}=>{native-country=United-States}']
rules

['{native-country=United-States,capital-gain=None}=>{capital-loss=None}',
 '{native-country=United-States,capital-loss=None}=>{capital-gain=None}',
 '{capital-gain=None,capital-loss=None}=>{native-country=United-States}']

In [85]:
nrows = df.shape[0]
def find_support(ls_rules):
    temp_df = df.copy()
    for l in ls_rules:
        column, value = l.split('=')
        temp_df = temp_df[temp_df[column] == value]
    return temp_df.shape[0]/nrows

def arrangingRules(rules):
    
    confidence_df = pd.DataFrame(columns=['Rule', 'Confidence'])
    for r in rules:
        a = r.split('=>')[0]
        b = r.split('=>')[1]

        a = re.sub('{', "", a)
        a = re.sub('}', "", a)

        b = re.sub('{', "", b)
        b = re.sub('}', "", b)

        ls_a = a.split(',')
        ls_b = b.split(',')

        support_a = find_support(ls_a)
        support_ab = find_support(ls_a + ls_b)

        confidence_atob = support_ab/support_a

        confidence_df = confidence_df.append({'Rule':r, 'Confidence':confidence_atob}, ignore_index = True)
    
    confidence_df = confidence_df.sort_values(by='Confidence', ascending = False)
    return list(confidence_df.Rule.values)

In [86]:
arrangingRules(rules)

['{native-country=United-States,capital-gain=None}=>{capital-loss=None}',
 '{native-country=United-States,capital-loss=None}=>{capital-gain=None}',
 '{capital-gain=None,capital-loss=None}=>{native-country=United-States}']